In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

trainData = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
testData = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
trainData.head()

In [ ]:
testData.head()

Here, we see that the test dataset is only useful for the competition organizers, but we cannot use it due to the absence of a "label" column. To address this, I will split the training dataset into training and testing portions.

In [ ]:
trainData = np.array(trainData)
rows, columns = trainData.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

print(f"rows: {rows} , columns: {columns}")

In [ ]:
testData = trainData[0:2000].T
testLabels = testData[0]
testPixels = testData[1:columns]
testPixels = testPixels / 255.

trainData = trainData[2000:rows].T
trainLabels = trainData[0]
trainPixels = trainData[1:columns]
trainPixels = trainPixels / 255.
_,rows_train = X_train.shape

In [ ]:
print(testLabels)
print(f"shape : {testLabels.shape}")

In [ ]:
print(testPixels)
print(f"shape: {testPixels.shape}")
print(f"shape: {testPixels[:,1].shape}")

In [ ]:
print(trainLabels)
print(f"shape : {trainLabels.shape}")

In [ ]:
print(trainPixels)
print(f"shape: {trainPixels.shape}")
print(f"shape: {trainPixels[:,1].shape}")

This all checks out so we can move on to building our network.

In [ ]:
def init_params():
    W1 = np.random.rand(10, 783) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [ ]:
def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(trainPixels, trainLabels, 0.10, 500)

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = trainPixels[:, index, None]
    prediction = make_predictions(trainPixels[:, index, None], W1, b1, W2, b2)
    label = trainLabels[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)